# Network Models: WS + BA

This notebook covers geospatial Watts-Strogatz (WS) and geospatial Barabasi-Albert (BA) models, including ordering effects for BA.


In [ ]:
import statistics
from collections import Counter

import contextily as cx
import geodatasets
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib_inline
import networkx as nx
import numpy as np
from matplotlib.ticker import MaxNLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable

from pysgn import geo_barabasi_albert_network, geo_watts_strogatz_network
from pysgn.ordering import attribute_order, density_order, random_order

matplotlib_inline.backend_inline.set_matplotlib_formats("svg")


def plot_degree_distribution(graph, ax=None):
    degree_sequence = sorted([d for _, d in graph.degree()], reverse=True)
    degree_count = Counter(degree_sequence)
    deg, cnt = zip(*degree_count.items())
    if ax is None:
        _, ax = plt.subplots(figsize=(12, 8))
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.set_xlim([-0.5, max(degree_sequence) + 1.5])
    ax.bar(deg, cnt, width=0.80, color="#43a2ca")
    ax.set_title(
        f"Network Degree Histogram ({graph.number_of_nodes():,} nodes)\n(mean={np.mean(degree_sequence):.1f}, std={np.std(degree_sequence):.2f}, mode={statistics.mode(degree_sequence)})",
    )
    ax.set_ylabel("Count")
    ax.set_xlabel("Degree")


We will reuse the same grocery-store dataset from Getting Started so this notebook can run independently.


In [ ]:
gdf = (
    gpd.read_file(geodatasets.get_path("geoda.groceries"))
    .explode(index_parts=False)
    .reset_index(drop=True)
    .to_crs("EPSG:26971")
)

_, ax = plt.subplots(figsize=(6, 6))
gdf.plot(ax=ax, markersize=6, color="#2b8cbe", alpha=0.8)
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, crs=gdf.crs)
ax.set_title("Grocery store locations (geodatasets)")
ax.set_axis_off()

# Example: Geospatial Watts-Strogatz Network

The geospatial Watts-Strogatz network provides more control on the resulting network, by allowing two more parameters:

- `k` (int or str): The number of nearest neighbors each node is initially connected to.
    * If an integer, it specifies the initial number of neighbors for all nodes. The resulting network has a mean degree of `k`.
    * If a string, it refers to a column in the GeoDataFrame that contains the expected degree centrality for each node.
- `p` (float): The probability of rewiring each edge. This parameter controls the randomness of the network. A value of 0 results in a regular lattice, while a value of 1 results in a random network.

First, the model connects each node to its `k` nearest neighbors.

Then, it rewires each edge with probability `p`. When an edge is rewired, it is removed and a new edge is added to a random node.
The probability of being rewired to a new node is determined by the distance between the nodes:
$$
  p(d|a, \textrm{min\_dist}) = \textrm{min}\left(1, \left(\frac{d}{\textrm{min\_dist}}\right) ^ {-a}\right)
$$
where `d` is edge length, `min_dist` is the minimum edge length, and `a` is the distance decay exponent parameter, default is 3.

This decay function of probabilities with respect to node distance works the same as above in the geospatial Erdős-Rényi network.

We'll first use a simple grid to visualize how `k` and `p` shape the resulting graph, then apply the model to the real grocery-store dataset introduced earlier.

In [ ]:
x = np.linspace(0, 10, 5)
y = np.linspace(0, 10, 5)
xx, yy = np.meshgrid(x, y)
points = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(xx.flatten(), yy.flatten()), crs="EPSG:3857"
)


k_values = [2, 6]
p_values = [0, 0.1, 0.5]
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

for i, k in enumerate(k_values):
    for j, p in enumerate(p_values):
        graph = geo_watts_strogatz_network(
            points, k=k, p=p, a=3, scaling_factor=0.05, random_state=42, verbose=False
        )
        ax = axes[i, j]
        pos = nx.get_node_attributes(graph, "pos")
        nx.draw(graph, pos=pos, node_size=50, ax=ax, with_labels=False)
        if p == 0:
            title = f"Local Connections Only: p={p}"
        elif p == 0.1:
            title = f"With Rewiring: p={p}"
        else:
            title = f"More Rewiring: p={p}"

        ax.set_title(f"\n{title}\n(k={k})", fontsize=22)

plt.tight_layout()

Now let's switch back to the grocery-store points, so we can see how the same parameters behave on real-world geometry:

In [ ]:
graph = geo_watts_strogatz_network(gdf, k=6, p=0.3)

_, ax = plt.subplots(1, 2, figsize=(10, 5))
gdf.plot(ax=ax[0], markersize=6, color="#b0b0b0", alpha=0.5)
nx.draw(
    graph,
    pos=nx.get_node_attributes(graph, "pos"),
    node_size=20,
    width=0.5,
    edge_color="#3182bd",
    node_color="#08519c",
    ax=ax[0],
)
cx.add_basemap(ax[0], source=cx.providers.CartoDB.Positron, crs=gdf.crs)
ax[0].set_title("Geospatial Watts-Strogatz Network\n(geoda.groceries)")
ax[0].set_axis_off()
plot_degree_distribution(graph, ax=ax[1])

As we can see, the average degree centrality is 6, the same as our `k` parameter.

Instead of setting a global `k` parameter for the average degree centrality of all nodes, it is possible to specify each node's expected degree individually. In this case, the `k` parameter can be set to the column name containing such information.

For instance, you may want to generate a synthetic geospatial network to represent people's social connections. From certain studies, you find out that people live in certain areas in general have higher degrees than those who live in other areas. Hence, you want to create a synthetic geospatial network that captures this information.

This is illustrated in the following example, where each node has an expected degree value based on its location.

In [ ]:
grid_size = 3
expected_degrees = np.array([[4, 4, 4], [4, 6, 4], [4, 6, 10]])

xmin, ymin, xmax, ymax = gdf.total_bounds
x_edges = np.linspace(xmin, xmax, grid_size + 1)
y_edges = np.linspace(ymin, ymax, grid_size + 1)


def assign_expected_degree(point, x_edges, y_edges, expected_degrees):
    x_bin = np.clip(np.digitize(point.x, x_edges) - 1, 0, grid_size - 1)
    y_bin = np.clip(grid_size - np.digitize(point.y, y_edges), 0, grid_size - 1)
    return expected_degrees[y_bin, x_bin]


gdf["expected_degree"] = gdf.geometry.apply(
    assign_expected_degree, args=(x_edges, y_edges, expected_degrees)
)

gdf.head()

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(6, 6))
divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="5%", pad=0.3)
gdf.plot(
    column="expected_degree",
    cmap="viridis_r",
    legend=True,
    cax=cax,
    legend_kwds={"label": "expected_degree", "orientation": "horizontal"},
    ax=ax,
    markersize=15,
    alpha=0.9,
)
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, crs=gdf.crs)
ax.set_title("Grocery locations colored by expected degree")
ax.set_axis_off()

We can then set `k` to be the `expected_degree` column:

In [ ]:
graph = geo_watts_strogatz_network(gdf, k="expected_degree", p=0.3)

degrees = dict(graph.degree())
cmap = plt.colormaps.get_cmap("viridis_r")
node_colors = [cmap(degree / max(degrees.values())) for node, degree in degrees.items()]

_, ax = plt.subplots(1, 2, figsize=(10, 5))
gdf.plot(ax=ax[0], markersize=4, color="#d9d9d9", alpha=0.4)
nx.draw(
    graph,
    pos=nx.get_node_attributes(graph, "pos"),
    node_color=node_colors,
    node_size=20,
    width=0.5,
    ax=ax[0],
)
cx.add_basemap(ax[0], source=cx.providers.CartoDB.Positron, crs=gdf.crs)
ax[0].set_title("Watts-Strogatz network with per-node expected degrees")
ax[0].set_axis_off()
plot_degree_distribution(graph, ax=ax[1])

As a result, nodes in the lower right corner have higher degrees (i.e., more connections) than nodes in other areas.

# Example: Geospatial Barabási-Albert Network

The geospatial Barabási-Albert network model combines preferential attachment with spatial constraints. In the original Barabási-Albert model, new nodes are more likely to connect to existing nodes with high degrees, leading to a "rich-get-richer" effect and a scale-free network structure.

In the geospatial variant, the probability of connection is influenced by both the node's degree and its geographic distance:

$$
  P(i) \propto k_i \cdot \textrm{min}\left(1, \left(\frac{\textrm{distance}}{\textrm{min\_dist}}\right) ^ {-a}\right)
$$

where:
- $k_i$ is the degree of existing node i
- $\textrm{distance}$ is the geographic distance between existing node i and the new node
- $\textrm{min\_dist}$ is the minimum distance threshold (1/scaling_factor)
- $a$ is the distance decay exponent parameter

The model has several key parameters:
- `m` (int): Number of edges to attach from a new node to existing nodes (and size of the seed network).
- `node_order` (callable or str): Order in which nodes are added to the network
- `a` (int): Distance decay exponent, controlling how strongly distance affects connection probability
- `scaling_factor` (float): Inverse of minimum distance threshold

Distance decay exponent `a` and parameter `scaling_factor` work the same as above in the geospatial Erdős-Rényi and Watts-Strogatz networks.

The `node_order` parameter controls the order in which nodes are added to the network. By default, nodes are added sequentially as they appear in the GeoDataFrame (first row is added first, second row is added second, and so on), but users can specify different ordering strategies:

- random order
- attribute order: Order by values in specific column(s) in ascending order
- density order (KNN or KDE): Order by spatial density (high to low)
- custom function: User-defined function that takes a GeoDataFrame and returns an order array

The sequence in which nodes join the network impacts the final structure. Early nodes typically become hubs with higher connectivity. Our implementation provides flexible ordering strategies based on spatial patterns, attributes, or custom logic. The choice of ordering strategy allows modeling various real-world network growth processes, such as urban expansion from dense centers or infrastructure development based on population needs.

Next, we'll reuse the grocery-store dataset but assign a few synthetic attributes (e.g., store "utility" or density ranking) so we can explore different node-ordering strategies with real coordinates.

In [ ]:
rng = np.random.default_rng(42)
gdf["utility"] = rng.integers(100, 10000, size=len(gdf))
points = np.column_stack((gdf.geometry.x.values, gdf.geometry.y.values))

gdf.head()

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(6, 6))
divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="5%", pad=0.3)
gdf.plot(
    column="utility",
    legend=True,
    cmap="viridis_r",
    cax=cax,
    legend_kwds={"label": "utility", "orientation": "horizontal"},
    ax=ax,
    markersize=20,
    alpha=0.9,
)
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, crs=gdf.crs)
ax.set_title("Grocery locations with synthetic 'utility'")
ax.set_axis_off()

Ordering functions used in the BA examples:
- `random_order`
- `attribute_order`
- `density_order`
- `density_order_knn` (`density_order(..., method="knn")`)
- `density_order_kde` (`density_order(..., method="kde")`)


Let's visualize different node orderings:

In [ ]:
# Apply different ordering strategies
random_idx = random_order(gdf, random_state=42)
utility_idx = attribute_order(gdf, "utility")
density_knn_idx = density_order(gdf, method="knn", k=5)
density_kde_idx = density_order(gdf, method="kde", bandwidth=0.5)

# Visualize different orderings
fig, axs = plt.subplots(2, 2, figsize=(13, 10))
titles = [
    "Random Order",
    "Attribute Order ('utility' column)",
    "Density Order (KNN)",
    "Density Order (KDE)",
]
orders = [random_idx, utility_idx, density_knn_idx, density_kde_idx]

for i, (ax, title, order) in enumerate(zip(axs.flatten(), titles, orders)):
    x, y = points[order, 0], points[order, 1]
    scatter = ax.scatter(x, y, c=np.arange(len(x)), cmap="viridis", s=80, alpha=0.9)
    ax.set_title(title)
    ax.set_axis_off()

fig.tight_layout()
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.8, 0.055, 0.02, 0.9])
cbar = fig.colorbar(scatter, cax=cbar_ax)
cbar.set_label("Node Addition Order")

We can create synthetic networks using each ordering strategy and compare the results.

In [ ]:
ordering_functions = [
    lambda frame: random_order(frame, random_state=42),
    lambda frame: attribute_order(frame, "utility"),
    lambda frame: density_order(frame, method="knn", k=5),
    lambda frame: density_order(frame, method="kde", bandwidth=0.5),
]

fig, axs = plt.subplots(2, 4, figsize=(18, 8))
for i, (title, order_fn) in enumerate(zip(titles, ordering_functions)):
    graph = geo_barabasi_albert_network(
        gdf,
        m=2,
        node_order=order_fn,
        random_state=42,
    )
    degrees = dict(graph.degree())
    cmap = plt.colormaps.get_cmap("viridis_r")
    node_colors = [
        cmap(degree / max(degrees.values())) for node, degree in degrees.items()
    ]
    gdf.plot(ax=axs[0, i], markersize=3, color="#cccccc", alpha=0.4)
    nx.draw(
        graph,
        pos=nx.get_node_attributes(graph, "pos"),
        node_color=node_colors,
        node_size=15,
        width=0.3,
        ax=axs[0, i],
        alpha=0.9,
    )
    cx.add_basemap(axs[0, i], source=cx.providers.CartoDB.Positron, crs=gdf.crs)
    axs[0, i].set_title(title)
    axs[0, i].set_axis_off()
    plot_degree_distribution(graph, axs[1, i])

The degree distribution plots show the characteristic power-law behavior of Barabási-Albert networks. The spatial constraints modify this pure power-law behavior, as geographic distance limits the potential connections. However, the network still maintains the general scale-free property, where a few hub nodes have many connections while most nodes have relatively few.

This model is particularly useful for modeling networks where:
1. The network grows over time
2. New nodes preferentially attach to well-connected nodes
3. Geographic distance influences connection probability
4. A scale-free degree distribution is expected

Examples include transportation networks, power grids, and some types of social networks where both popularity and proximity affect connection patterns.

## WS vs BA: quick comparison notes

- WS starts from local neighbor structure and rewires edges with probability `p`, so clustering remains comparatively strong for lower `p`.
- BA grows the graph sequentially, and early-added nodes can become hubs through preferential attachment.
- Spatial decay affects both models by discouraging long-distance edges, but BA still tends to show more hub concentration.
- Ordering choices in BA (`random_order`, attribute-based, density-based) can materially change hub placement and degree spread.


## See also

- Previous: [Getting Started](getting_started.ipynb)
- Next: [Utilities](utils.ipynb)
- Advanced topics: [Advanced Options and Performance](advanced_options_performance.ipynb)
